In [1]:
import torch
import pandas as pd
import polars as pl
import nannyml as nml

from nannyml.thresholds import ConstantThreshold

In [2]:
cur_df = pl.read_csv('data/preprocessed/train.csv')
old_df = pl.read_csv('data/preprocessed/val.csv')

# Add noise to prediction to simulate prediction drift
cur_df = cur_df.with_columns(pl.col('Pawpularity').add(3).alias('Prediction')).to_pandas()
old_df = old_df.with_columns(pl.col('Pawpularity').add(5).alias('Prediction')).to_pandas()

In [3]:
# All columns except image id
all_cols = [col for col in cur_df.columns if col != 'Id']
# Target and prediction
cont_cols = ['Pawpularity', 'Prediction']
# Binary features, can be treated as categorical or continuous
bin_cols = [col for col in all_cols if col not in cont_cols]

In [4]:
calc = nml.UnivariateDriftCalculator(
    column_names = all_cols,
    # treat_as_categorical = bin_cols,
    continuous_methods = ['jensen_shannon'],
    # categorical_methods = ['jensen_shannon'],
    chunk_size = 100,
    thresholds = {
        # Jensen-Shannon score is not the same as p-value
        # 0 means identical data and 1 means very different
        'jensen_shannon': ConstantThreshold(upper = 0.1)
    }
)

calc.fit(old_df)
drift_result = calc.calculate(cur_df)

with pd.option_context('display.max_columns', None): 
    display(drift_result.to_df(multilevel = True))

chunk                                                                   \
       chunk                                                                    
         key chunk_index start_index end_index start_date end_date     period   
0     [0:99]           0           0        99       None     None  reference   
1  [100:199]           1         100       199       None     None  reference   
2     [0:99]           0           0        99       None     None   analysis   
3  [100:199]           1         100       199       None     None   analysis   
4  [200:299]           2         200       299       None     None   analysis   
5  [300:399]           3         300       399       None     None   analysis   
6  [400:499]           4         400       499       None     None   analysis   
7  [500:599]           5         500       599       None     None   analysis   
8  [600:699]           6         600       699       None     None   analysis   
9  [700:799]           7         700       799       None     None   analysis   

       Accessory                                                Action  \
  jensen_shannon                                        jensen_shannon   
           value upper_threshold lower_threshold  alert          value   
0       0.053003             0.1            None  False       0.024846   
1       0.042430             0.1            None  False       0.050045   
2       0.057581             0.1            None  False       0.059304   
3       0.098767             0.1            None  False       0.050045   
4       0.057581             0.1            None  False       0.024846   
5       0.098767             0.1            None  False       0.085208   
6       0.057581             0.1            None  False       0.050045   
7       0.009522             0.1            None  False       0.050045   
8       0.053003             0.1            None  False       0.024846   
9       0.042430             0.1            None  False       0.024846   

                                                   Blur                  \
                                         jensen_shannon                   
  upper_threshold lower_threshold  alert          value upper_threshold   
0             0.1            None  False       0.008464             0.1   
1             0.1            None  False       0.008773             0.1   
2             0.1            None  False       0.008773             0.1   
3             0.1            None  False       0.039785             0.1   
4             0.1            None  False       0.027399             0.1   
5             0.1            None  False       0.024585             0.1   
6             0.1            None  False       0.047818             0.1   
7             0.1            None  False       0.047818             0.1   
8             0.1            None  False       0.024585             0.1   
9             0.1            None  False       0.024585             0.1   

                                Collage                                  \
                         jensen_shannon                                   
  lower_threshold  alert          value upper_threshold lower_threshold   
0            None  False       0.011980             0.1            None   
1            None  False       0.011180             0.1            None   
2            None  False       0.031656             0.1            None   
3            None  False       0.011180             0.1            None   
4            None  False       0.067251             0.1            None   
5            None  False       0.031656             0.1            None   
6            None  False       0.073625             0.1            None   
7            None  False       0.067251             0.1            None   
8            None  False       0.050193             0.1            None   
9            None  False       0.050193             0.1            None   

                   Eyes               

In [5]:
with pd.option_context('display.max_columns', None): 
    test = drift_result.filter(period = 'analysis').to_df(multilevel = True)
    # If True more than 30% (0.3) then assume the data drifted
    display(test['Accessory','jensen_shannon','alert'].value_counts(normalize = True).get(True, 0))

0

In [6]:
fig = drift_result.plot(kind = 'drift')
fig.update_layout(width = 800)
fig.show()

In [7]:
fig = drift_result.plot(kind = 'distribution')
fig.update_layout(width = 800)
fig.show()

Red color means drift is detected, while purple is considered normal (no drift)

Now let's see the performance calculator

In [8]:
calc = nml.PerformanceCalculator(
    metrics = ['rmse'],
    problem_type = 'regression',
    y_pred = 'Prediction',
    y_true = 'Pawpularity',
    chunk_size = 100,
    thresholds = {
        # Pawpularity range is from 0 to 100
        # Error of 25 is bad enough, or even very bad
        'rmse': ConstantThreshold(upper = 25)
    }
)

calc.fit(old_df)
perf_result = calc.calculate(cur_df)

with pd.option_context('display.max_columns', None): 
    display(perf_result.to_df(multilevel = True))

chunk                                                                   \
         key chunk_index start_index end_index start_date end_date     period   
0     [0:99]           0           0        99       None     None  reference   
1  [100:199]           1         100       199       None     None  reference   
2     [0:99]           0           0        99       None     None   analysis   
3  [100:199]           1         100       199       None     None   analysis   
4  [200:299]           2         200       299       None     None   analysis   
5  [300:399]           3         300       399       None     None   analysis   
6  [400:499]           4         400       499       None     None   analysis   
7  [500:599]           5         500       599       None     None   analysis   
8  [600:699]           6         600       699       None     None   analysis   
9  [700:799]           7         700       799       None     None   analysis   

                                 rmse                                        \
  targets_missing_rate sampling_error value upper_threshold lower_threshold   
0                  0.0            0.0   5.0              25            None   
1                  0.0            0.0   5.0              25            None   
2                  0.0            0.0   3.0              25            None   
3                  0.0            0.0   3.0              25            None   
4                  0.0            0.0   3.0              25            None   
5                  0.0            0.0   3.0              25            None   
6                  0.0            0.0   3.0              25            None   
7                  0.0            0.0   3.0              25            None   
8                  0.0            0.0   3.0              25            None   
9                  0.0            0.0   3.0              25            None   

          
   alert  
0  False  
1  False  
2  False  
3  False  
4  False  
5  False  
6  False  
7  False  
8  False  
9  False

In [9]:
fig = perf_result.plot(kind = 'performance')
fig.update_layout(width = 800)
fig.show()

Now let's see the rank

In [10]:
ranker = nml.CorrelationRanker()
ranker.fit(perf_result.filter(period = 'reference', metrics = ['rmse']))

rank_df = ranker.rank(
    drift_result.filter(period = 'analysis', metrics = ['jensen_shannon']),
    perf_result.filter(period = 'analysis', metrics = ['rmse']),
    only_drifting = False
)

rank_df

c:\Users\Dhika\Apps\CommonFiles\Python\envs\pytorch\Lib\site-packages\nannyml\drift\ranker.py:376: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,Accessory,NaN,NaN,False,1
1,Action,NaN,NaN,False,2
2,Blur,NaN,NaN,False,3
3,Collage,NaN,NaN,False,4
4,Eyes,NaN,NaN,True,5
5,Face,NaN,NaN,True,6
6,Group,NaN,NaN,False,7
7,Human,NaN,NaN,True,8
8,Info,NaN,NaN,False,9
9,Near,NaN,NaN,False,10


This rank is not usable, sometimes it's nan or constantly at 1.0

Currently I don't have time to debug this, so we will just make our own table later as a workaround